In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'31u50c1e'

In [3]:
wandb.init(id='31u50c1e', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14029.957

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 1min 4s, sys: 2.23 s, total: 1min 7s
Wall time: 1min 7s


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [9]:
"""for n, p in m.named_parameters():
    if 'fcs_2' in n:
        p.requires_grad = True
    else:
        p.requires_grad = False
        
    print(n, p.requires_grad)"""

"for n, p in m.named_parameters():\n    if 'fcs_2' in n:\n        p.requires_grad = True\n    else:\n        p.requires_grad = False\n        \n    print(n, p.requires_grad)"

In [10]:
model_stem = "9.19"
START_E = 0

In [11]:
n_epochs = 10000
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = 6e-4 if e < 8 else 1e-4

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 0

{'trn_control_loss': 0.02378106, 'trn_headings_loss': 0.00113714, 'trn_curvatures_loss': 0.00470673, 'consistency losses/trn_steer_cost': 0.00641863, 'consistency losses/trn_te_loss': 20.51302715, 'aux losses/trn_pitch_loss': 0.31335926, 'aux losses/trn_yaw_loss': 0.22849369, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.75, 'consistency losses/trn_torque_loss': 2826.92189407, 'consistency losses/trn_torque_delta_loss': 14.20090522, 'logistical/obs_consumed_per_second': 199.81102362, 'logistical/obs_generated_per_second': 167.66141732, 'logistical/data_consumption_ratio': 1.19207207, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.29717827, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 200.171875, 'logistical/obs_generated_per_second': 163.078125, 'logistical/data_consumption_ratio': 1.22878763, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02502912, 'trn_headings_loss': 0.0010756, 'trn_curvatures_loss': 0.0038254, '

{'logistical/obs_consumed_per_second': 197.6796875, 'logistical/obs_generated_per_second': 170.125, 'logistical/data_consumption_ratio': 1.16240968, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02275109, 'trn_headings_loss': 0.0009158, 'trn_curvatures_loss': 0.00208244, 'consistency losses/trn_steer_cost': 0.00421858, 'consistency losses/trn_te_loss': 22.38433045, 'aux losses/trn_pitch_loss': 0.22298765, 'aux losses/trn_yaw_loss': 0.17139196, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.890625, 'consistency losses/trn_torque_loss': 3076.59922028, 'consistency losses/trn_torque_delta_loss': 17.30542643, 'logistical/max_param': 16.2802372, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 196.2578125, 'logistical/obs_generated_per_second': 168.1640625, 'logistical/data_consumption_ratio': 1.16796671, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02438286, 'trn_headings_loss': 0.00107178, 'trn_curvatures_loss': 0.00365994, 'consisten

{'logistical/obs_consumed_per_second': 200.40625, 'logistical/obs_generated_per_second': 167.421875, 'logistical/data_consumption_ratio': 1.1976508, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.03522688, 'trn_headings_loss': 0.00137457, 'trn_curvatures_loss': 0.00396302, 'consistency losses/trn_steer_cost': 0.00614566, 'consistency losses/trn_te_loss': 21.85803695, 'aux losses/trn_pitch_loss': 0.19940138, 'aux losses/trn_yaw_loss': 0.22511816, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.8671875, 'consistency losses/trn_torque_loss': 2615.8355608, 'consistency losses/trn_torque_delta_loss': 17.2062216, 'logistical/max_param': 16.26222229, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 201.8046875, 'logistical/obs_generated_per_second': 164.828125, 'logistical/data_consumption_ratio': 1.22383817, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02560055, 'trn_headings_loss': 0.00125875, 'trn_curvatures_loss': 0.00331444, 'consisten

{'logistical/obs_consumed_per_second': 202.6875, 'logistical/obs_generated_per_second': 166.75, 'logistical/data_consumption_ratio': 1.21626625, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01967549, 'trn_headings_loss': 0.0008563, 'trn_curvatures_loss': 0.0029673, 'consistency losses/trn_steer_cost': 0.00625515, 'consistency losses/trn_te_loss': 20.6107043, 'aux losses/trn_pitch_loss': 0.17247534, 'aux losses/trn_yaw_loss': 0.22579336, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.7890625, 'consistency losses/trn_torque_loss': 2635.72273397, 'consistency losses/trn_torque_delta_loss': 14.246991, 'logistical/max_param': 16.29168129, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 200.1953125, 'logistical/obs_generated_per_second': 170.6875, 'logistical/data_consumption_ratio': 1.17274395, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02535266, 'trn_headings_loss': 0.00119546, 'trn_curvatures_loss': 0.0051715, 'consistency losses/

{'logistical/obs_consumed_per_second': 199.4453125, 'logistical/obs_generated_per_second': 164.625, 'logistical/data_consumption_ratio': 1.21237838, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01627338, 'trn_headings_loss': 0.00077802, 'trn_curvatures_loss': 0.00223958, 'consistency losses/trn_steer_cost': 0.00493507, 'consistency losses/trn_te_loss': 21.14636301, 'aux losses/trn_pitch_loss': 0.14240122, 'aux losses/trn_yaw_loss': 0.20371962, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.875, 'consistency losses/trn_torque_loss': 2930.81247091, 'consistency losses/trn_torque_delta_loss': 13.55729154, 'logistical/max_param': 16.2721405, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 197.5, 'logistical/obs_generated_per_second': 166.5625, 'logistical/data_consumption_ratio': 1.18531024, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02096719, 'trn_headings_loss': 0.00109563, 'trn_curvatures_loss': 0.00277555, 'consistency losses/t

...

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 149, in check_network_status
    status_response = self._interface.communicate_network_status()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 120, in communicate_network_status
    resp = self._communicate_network_status(status)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface_queue.py", line 411, in _communicate_network_status
    resp = self._communicate(req, local=True)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface

Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2140985/2466313303.py", line 10, in <module>
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler,
  File "/home/beans/bespoke/train_utils.py", line 192, in run_epoch
    batch = dataloader.get_batch()
  File "/home/beans/bespoke/blender_dataloader.py", line 162, in get_batch
    time.sleep(.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most rec

TypeError: object of type 'NoneType' has no len()

Error in callback <function _WandbInit._pause_backend at 0x7fc95a4540d0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [12]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e58.torch")

Error in callback <function _WandbInit._resume_backend at 0x7fc994177ee0> (for pre_run_cell):


Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x7fc95a4540d0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [13]:
set_should_stop(True)

Error in callback <function _WandbInit._resume_backend at 0x7fc994177ee0> (for pre_run_cell):


Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x7fc95a4540d0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
